<a href="https://colab.research.google.com/github/hpayettepeterson/im2recipe-Pytorch/blob/master/testing_im2recipe_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up notebook

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# enter correct folder
%cd /content/drive/MyDrive/Colab_Notebooks/foodprint.ai

/content/drive/MyDrive/Colab_Notebooks/foodprint.ai


In [ ]:
# install requirements
!pip install -q -r ./requirements.txt
!pip install -q torchwordemb

     |████████████████████████████████| 42 kB 675 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 129 kB 9.3 MB/s 


# Load model

In [ ]:
# try loading model to see if it works...

import torch
import torchwordemb

from trijoint import im2recipe


if not(torch.cuda.device_count()):
    device = torch.device(*('cpu',0))
else:
    torch.cuda.manual_seed(opts.seed)
    device = torch.device(*('cuda',0))

#import ipdb; ipdb.set_trace()
model = im2recipe()
model.visionMLP = torch.nn.DataParallel(model.visionMLP)
model.to(device)
model_path=  "snapshots/model_e220_v-4.700.pth.tar"
...
checkpoint = torch.load(model_path, encoding='latin1', map_location='cpu')
...
if device.type=='cpu':
  checkpoint = torch.load(model_path, encoding='latin1', map_location='cpu')
else:
  checkpoint = torch.load(model_path, encoding='latin1')
model.load_state_dict(checkpoint['state_dict'])

As is, the above code seems to work to load the model

# Preparing recipes_with_nutritional_info to extract embeddings

## What are the recipe datasets like?

layer1 and recipes_with_nutritional_info are both json files with recipe info.
The difference between them is the extent of info contained. Below are the keys for each dictionary:

In [ ]:
nutritional_dict = {"fsa_lights_per100g":{"fat":"green","salt":"green","saturates":"green","sugars":"orange"},"id":"000095fc1d","ingredients":[{"text":"yogurt, greek, plain, nonfat"},{"text":"strawberries, raw"},{"text":"cereals ready-to-eat, granola, homemade"}],"instructions":[{"text":"Layer all ingredients in a serving dish."}],"nutr_per_ingredient":[{"fat":0.8845044000000001,"nrg":133.80964,"pro":23.110512399999998,"sat":0.26535132,"sod":81.64656,"sug":7.348190400000001},{"fat":0.46,"nrg":49.0,"pro":1.02,"sat":0.023,"sod":2.0,"sug":7.43},{"fat":7.415,"nrg":149.25,"pro":4.17,"sat":1.207,"sod":8.0,"sug":6.04}],"nutr_values_per100g":{"energy":81.129461318947662,"fat":2.1401392635158909,"protein":6.9144365935655356,"salt":0.055978167389859668,"saturates":0.36534716195613937,"sugars":5.0863410343614399},"partition":"train","quantity":[{"text":"8"},{"text":"1"},{"text":"1/4"}],"title":"Yogurt Parfaits","unit":[{"text":"ounce"},{"text":"cup"},{"text":"cup"}],"url":"http://tastykitchen.com/recipes/breakfastbrunch/yogurt-parfaits/","weight_per_ingr":[226.796,152.0,30.5]}
layer_dict1 = {"ingredients": [{"text": "6 ounces penne"}, {"text": "2 cups Beechers Flagship Cheese Sauce (recipe follows)"}, {"text": "1 ounce Cheddar, grated (1/4 cup)"}, {"text": "1 ounce Gruyere cheese, grated (1/4 cup)"}, {"text": "1/4 to 1/2 teaspoon chipotle chili powder (see Note)"}, {"text": "1/4 cup (1/2 stick) unsalted butter"}, {"text": "1/3 cup all-purpose flour"}, {"text": "3 cups milk"}, {"text": "14 ounces semihard cheese (page 23), grated (about 3 1/2 cups)"}, {"text": "2 ounces semisoft cheese (page 23), grated (1/2 cup)"}, {"text": "1/2 teaspoon kosher salt"}, {"text": "1/4 to 1/2 teaspoon chipotle chili powder"}, {"text": "1/8 teaspoon garlic powder"}, {"text": "(makes about 4 cups)"}], "url": "http://www.epicurious.com/recipes/food/views/-world-s-best-mac-and-cheese-387747", "partition": "train", "title": "Worlds Best Mac and Cheese", "id": "000018c8a5", "instructions": [{"text": "Preheat the oven to 350 F. Butter or oil an 8-inch baking dish."}, {"text": "Cook the penne 2 minutes less than package directions."}, {"text": "(It will finish cooking in the oven.)"}, {"text": "Rinse the pasta in cold water and set aside."}, {"text": "Combine the cooked pasta and the sauce in a medium bowl and mix carefully but thoroughly."}, {"text": "Scrape the pasta into the prepared baking dish."}, {"text": "Sprinkle the top with the cheeses and then the chili powder."}, {"text": "Bake, uncovered, for 20 minutes."}, {"text": "Let the mac and cheese sit for 5 minutes before serving."}, {"text": "Melt the butter in a heavy-bottomed saucepan over medium heat and whisk in the flour."}, {"text": "Continue whisking and cooking for 2 minutes."}, {"text": "Slowly add the milk, whisking constantly."}, {"text": "Cook until the sauce thickens, about 10 minutes, stirring frequently."}, {"text": "Remove from the heat."}, {"text": "Add the cheeses, salt, chili powder, and garlic powder."}, {"text": "Stir until the cheese is melted and all ingredients are incorporated, about 3 minutes."}, {"text": "Use immediately, or refrigerate for up to 3 days."}, {"text": "This sauce reheats nicely on the stove in a saucepan over low heat."}, {"text": "Stir frequently so the sauce doesnt scorch."}, {"text": "This recipe can be assembled before baking and frozen for up to 3 monthsjust be sure to use a freezer-to-oven pan and increase the baking time to 50 minutes."}, {"text": "One-half teaspoon of chipotle chili powder makes a spicy mac, so make sure your family and friends can handle it!"}, {"text": "The proportion of pasta to cheese sauce is crucial to the success of the dish."}, {"text": "It will look like a lot of sauce for the pasta, but some of the liquid will be absorbed."}]}

In [ ]:
# DIFFERENCE BETWEEN layer1 AND recipes_with_nutritional_info
print(layer_dict1.keys())
print(nutritional_dict.keys())

dict_keys(['ingredients', 'url', 'partition', 'title', 'id', 'instructions'])
dict_keys(['fsa_lights_per100g', 'id', 'ingredients', 'instructions', 'nutr_per_ingredient', 'nutr_values_per100g', 'partition', 'quantity', 'title', 'unit', 'url', 'weight_per_ingr'])


The keys we will need to output from recipes_with_nutritional_info are: 'ingredients' and 'weight_per_ingr'. We might also want other info...

It seems like recipes_with_nutritional_info is in a similar enough format to layer1 that I could feed it into the tokenizer as-is. I just need to figure out how to do that.

## Trying to tokenize the recipes

**NOTE** I moved tokenize_instructions from scripts folder to base folder (foodprint.ai) because I didn't know how to import it otherwise

### readfile() and tok()

Below I'm running the 2 functions, readfile and tok, from tokenize_instructions to see how they work on their own

- readfile: opens file, strips trailing spaces, returns list of strings, 1 line = 1 item. Readfile seems to be designed to work with layer1 and not recipes_with_nutritional_info
- tok: replaces punctuation with spaces

In [ ]:
# read file using function from tokenize_instructions
from tokenize_instructions import readfile, tok

file = "data/recipes_with_nutritional_info.json"
lines = readfile(file)

In [ ]:
line = lines[7]
type(line), line

(str, '        },')

As seen above, readfile applied to recipes_with_nutritional_info outputs data in a strange format

In [ ]:
file2 = "data/layer1.json"
lines2 = readfile(file2)

In [ ]:
line = lines2[7]

type(line), line

(str,
 '{"ingredients": [{"text": "2 Chicken thighs"}, {"text": "2 tsp Kombu tea"}, {"text": "1 White pepper"}], "url": "https://cookpad.com/us/recipes/150100-kombu-tea-grilled-chicken-thigh", "partition": "train", "title": "Kombu Tea Grilled Chicken Thigh", "id": "000075604a", "instructions": [{"text": "Pierce the skin of the chicken with a fork or knife."}, {"text": "Sprinkle with kombu tea evenly on both sides of the chicken, about 1 teaspoon per chicken thigh."}, {"text": "Brown the skin side of the chicken first over high heat until golden brown."}, {"text": "Sprinkle some pepper on the meat just before flipping over."}, {"text": "Then brown the other side until golden brown."}]},')

Above, readfile applied to layer1 seems to output data in a nicer format. I'm not sure why the difference

In [ ]:
# run function tok from tokenize_instructions

tokenized_text = tok(line)
  
tokenized_text

'{ " ingredients "  :  [{ " text "  :   " 2 Chicken thighs " } ,  { " text "  :   " 2 tsp Kombu tea " } ,  { " text "  :   " 1 White pepper " }] ,   " url "  :   " https : //cookpad . com/us/recipes/150100-kombu-tea-grilled-chicken-thigh "  ,   " partition "  :   " train "  ,   " title "  :   " Kombu Tea Grilled Chicken Thigh "  ,   " id "  :   " 000075604a "  ,   " instructions "  :  [{ " text "  :   " Pierce the skin of the chicken with a fork or knife .  " } ,  { " text "  :   " Sprinkle with kombu tea evenly on both sides of the chicken ,  about 1 teaspoon per chicken thigh .  " } ,  { " text "  :   " Brown the skin side of the chicken first over high heat until golden brown .  " } ,  { " text "  :   " Sprinkle some pepper on the meat just before flipping over .  " } ,  { " text "  :   " Then brown the other side until golden brown .  " }]} , '

tok() takes out punctuation from a string and replaces it with spaces

### the rest of tokenize_instructions

Below I am running the code from tokenize_instructions. It takes the recipe data and puts it into one text file

In [ ]:
# I changed file paths here so it would find and make files in correct places

import json
import numpy as np
import pickle
import sys
from tqdm import *
import time

def readfile(filename):
    with open(filename,'r') as f:
        lines = []
        for line in f.readlines():
            lines.append(line.rstrip())
    return lines

def tok(text,ts=False):

    '''
    Usage: tokenized_text = tok(text,token_list)
    If token list is not provided default one will be used instead.
    '''

    if not ts:
        ts = [',','.',';','(',')','?','!','&','%',':','*','"']

    for t in ts:
        text = text.replace(t,' ' + t + ' ')
    return text

if __name__ == "__main__":

    '''
    Generate tokenized text for w2v training
    Words separated with ' '
    Different instructions separated with \t
    Different recipes separated with \n
    '''

    try:
        partition = 'test' # HERE YOU NEED TO SPECIFY THE PARTITION
    except:
        partition = ''

    dets = json.load(open('data/det_ingrs.json','r')) # ingredients database
    
    # here you can change the file that is tokenized (was originally layer1)
    layer1 = json.load(open('data/recipes_with_nutritional_info.json','r')) 

    idx2ind = {}
    ingrs = []
    for i,entry in enumerate(dets):
        idx2ind[entry['id']] = i


    t = time.time()
    print("Saving tokenized here:", 'data/tokenized_instructions_'+partition+'.txt')
    f = open('data/tokenized_instructions_'+partition+'.txt','w')
    for i,entry in tqdm(enumerate(layer1)):
        '''
        if entry['id'] in dups:
            continue
        '''
        if not partition=='' and not partition == entry['partition']:
            continue
        instrs = entry['instructions']

        allinstrs = ''
        for instr in instrs:
            instr =  instr['text']
            allinstrs+=instr + '\t'

        # find corresponding set of detected ingredients
        det_ingrs = dets[idx2ind[entry['id']]]['ingredients']
        valid = dets[idx2ind[entry['id']]]['valid']

        for j,det_ingr in enumerate(det_ingrs):
            # if detected ingredient matches ingredient text,
            # means it did not work. We skip
            if not valid[j]:
                continue
            # underscore ingredient

            det_ingr_undrs = det_ingr['text'].replace(' ','_')
            ingrs.append(det_ingr_undrs)
            allinstrs = allinstrs.replace(det_ingr['text'],det_ingr_undrs)

        f.write(allinstrs + '\n')

    f.close()
    print(time.time() - t, 'seconds.')
    print("Number of unique ingredients",len(np.unique(ingrs)))
    f = open('data/tokenized_instructions_'+partition+'.txt','r')
    text = f.read()
    text = tok(text)
    f.close()

    f = open('data/tokenized_instructions_'+partition+'.txt','w')
    f.write(text)
    f.close()


Saving tokenized here: data/tokenized_instructions_test.txt


51235it [00:00, 209330.74it/s]


0.271381139755249 seconds.
Number of unique ingredients 2838


Need to run above code specifying the partition as "train", "val" or "test"

# Word2Vec

Need to get word2vec to work (below code) and run it on the file created by tokenize_instructions. This will tokenize the recipe data so we can feed it into recipe2embedding. Need to use their custom word2vec which can be downloaded on github

In [ ]:
# need to change code so it runs with tokenized_instructions_all
./word2vec -hs 1 -negative 0 -window 10 -cbow 0 -iter 10 -size 300 -binary 1 -min-count 10 -threads 20 -train tokenized_instructions_train.txt -output vocab.bin


# Recipe2Embedding

The following is code adapted from recipe2embedding. Once we have tokenized the recipe data, we can feed in the proper format of y1, y2, z1 and z2 to test the model

In [ ]:
import torch
import torchwordemb

from trijoint import recipe2emb
from args import get_parser

parser = get_parser()
opts = parser.parse_args()

if not(torch.cuda.device_count()):
    device = torch.device(*('cpu',0))
else:
    torch.cuda.manual_seed(opts.seed)
    device = torch.device(*('cuda',0))

#import ipdb; ipdb.set_trace()
recipeEmbModel = recipe2emb()
recipeEmbModel.to(device)
model_path=  "snapshots/model_e220_v-4.700.pth.tar"

# load checkpoint
print("=> loading checkpoint '{}'".format(opts.model_path))
if device.type=='cpu':
    checkpoint = torch.load(opts.model_path, encoding='latin1', map_location='cpu')
else:
    checkpoint = torch.load(opts.model_path, encoding='latin1')
opts.start_epoch = checkpoint['epoch']
print("=> loaded checkpoint '{}' (epoch {})"
      .format(opts.model_path, checkpoint['epoch']))

output = model(y1, y2, z1, z2)
output

=> loading checkpoint 'snapshots/model_e220_v-4.700.pth.tar'
=> loaded checkpoint 'snapshots/model_e220_v-4.700.pth.tar' (epoch 220)


NameError: ignored

# What else do I need to do???

- use word2vec to tokenize text from tokenize_instructions_all.txt file
- feed tokenized data into recipe2embedding as y1, y2, z1, z2
- debug recipe2embedding
- save recipe embeddings
- get image2embedding to work
- take simplified code & model and set it up so it can run on a computer and then on the cloud

In [ ]:
# check test.tar for data format
# figure out the format that recipe2embedding takes (I think it needs tokenized data)
# extract all recipe embeddings from this data and save in dictionary with key=embedding, value=ingredients, quantities etc

# Image to embedding model

In [ ]:
import json

recipe_path = 'data/recipes_with_nutritional_info.json'
with open(recipe_path, 'r') as f:
    recipes = json.load(f)

recipes

[{'fsa_lights_per100g': {'fat': 'green',
   'salt': 'green',
   'saturates': 'green',
   'sugars': 'orange'},
  'id': '000095fc1d',
  'ingredients': [{'text': 'yogurt, greek, plain, nonfat'},
   {'text': 'strawberries, raw'},
   {'text': 'cereals ready-to-eat, granola, homemade'}],
  'instructions': [{'text': 'Layer all ingredients in a serving dish.'}],
  'nutr_per_ingredient': [{'fat': 0.8845044000000001,
    'nrg': 133.80964,
    'pro': 23.110512399999998,
    'sat': 0.26535132,
    'sod': 81.64656,
    'sug': 7.348190400000001},
   {'fat': 0.46,
    'nrg': 49.0,
    'pro': 1.02,
    'sat': 0.023,
    'sod': 2.0,
    'sug': 7.43},
   {'fat': 7.415,
    'nrg': 149.25,
    'pro': 4.17,
    'sat': 1.207,
    'sod': 8.0,
    'sug': 6.04}],
  'nutr_values_per100g': {'energy': 81.12946131894766,
   'fat': 2.140139263515891,
   'protein': 6.914436593565536,
   'salt': 0.05597816738985967,
   'saturates': 0.36534716195613937,
   'sugars': 5.08634103436144},
  'partition': 'train',
  'quanti

In [ ]:
keys = recipes[10].keys()

dict_keys

In [ ]:
entry = recipes[9]
entry['partition'], entry['id']

('test', '0009785a2e')

In [ ]:
recipes

In [ ]:
import numpy as np

keys_list = []
for i in range(len(layer1)):
  for key in layer1[i].keys():
    keys_list.append(key)

def unique(list1):
    x = np.array(list1)
    print(np.unique(x))

unique(keys_list)

['id' 'ingredients' 'instructions' 'partition' 'title' 'url']


In [ ]:
# get image from URL

from PIL import Image
import urllib.request
import io

URL = 'https://www.licious.in/blog/wp-content/uploads/2020/12/3-Step-Chicken-Salad.jpg'

with urllib.request.urlopen(URL) as url:
    f = io.BytesIO(url.read())

img = Image.open(f)

In [ ]:

import time
import torch
import torch.nn as nn
import torch.nn.parallel
# import torch.optim
# import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.backends.cudnn as cudnn
# from data_loader import ImagerLoader # our data_loader
import numpy as np
from trijoint import im2recipe
import pickle
from args import get_parser
from PIL import Image
import sys
import os

# =============================================================================
parser = get_parser()
opts = parser.parse_args()
# =============================================================================

if not(torch.cuda.device_count()):
    device = torch.device(*('cpu',0))
else:
    torch.cuda.manual_seed(opts.seed)
    device = torch.device(*('cuda',0))

def norm(input, p=2, dim=1, eps=1e-12):
    return input / input.norm(p,dim,keepdim=True).clamp(min=eps).expand_as(input)

def main():
   
    # im_path = 'data/17-minute-caesar-salad_1980x1320-140424-1.jpeg'
    # ext = os.path.basename(im_path).split('.')[-1]
    # if ext not in ['jpeg','jpg','png']:
    #     raise Exception("Wrong image format.")

    # create model
    model = im2recipe()
    model.visionMLP = torch.nn.DataParallel(model.visionMLP)
    model.to(device)

    # load checkpoint
    print("=> loading checkpoint '{}'".format(opts.model_path))
    if device.type=='cpu':
        checkpoint = torch.load(opts.model_path, encoding='latin1', map_location='cpu')
    else:
        checkpoint = torch.load(opts.model_path, encoding='latin1')
    opts.start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    print("=> loaded checkpoint '{}' (epoch {})"
          .format(opts.model_path, checkpoint['epoch']))

    # data preparation, loaders
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([
                transforms.Scale(256), # rescale the image keeping the original aspect ratio
                transforms.CenterCrop(224), # we get only the center of that rescaled
                transforms.ToTensor(),
                normalize])

    # load image
    #im = Image.open(im_path).convert('RGB')
    im = img
    im = transform(im)
    im = im.view((1,)+im.shape)
    # get model output
    output = model.visionMLP(im)
    output = norm(output)
    output = output.data.cpu().numpy()
    # save output
    return output

embedding = main()
embedding

=> loading checkpoint 'snapshots/model_e220_v-4.700.pth.tar'
=> loaded checkpoint 'snapshots/model_e220_v-4.700.pth.tar' (epoch 220)


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:310: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


array([[[[0.01964093]],

        [[0.0176684 ]],

        [[0.02993617]],

        ...,

        [[0.00937774]],

        [[0.02176086]],

        [[0.02307766]]]], dtype=float32)

In [ ]:
len(embedding[0])

2048

In [ ]:
recipe_path = 'data/layer1.json'
with open(recipe_path, 'r') as f:
    layer1 = json.load(f)
len(layer1)

1029720

In [ ]:

entry = layer1[3]
print(entry['partition'], entry['id'])

test 00003a70b1


# What to do

- extract images from images_with_nutritional_info json
- run images through image2embedding model, save embeddings (can save to dict like we did with ingredient embeddings)
- plot embeddings
- map img embeddings to ingredient embeddings

In [ ]:
# rank.py
import os
import random
import numpy as np
import utils
import torchfile
import pickle
import sys
sys.path.append("..")
from args import get_parser

# =============================================================================
parser = get_parser()
opts = parser.parse_args()
# =============================================================================

random.seed(opts.seed)
type_embedding = 'image'
# 'image'
# type_embedding = 'recipe'
print(opts.path_results)
with open(os.path.join(opts.path_results,'img_embeds.pkl'),'rb') as f:
    im_vecs = pickle.load(f)
with open(os.path.join(opts.path_results,'rec_embeds.pkl'),'rb') as f:
    instr_vecs = pickle.load(f)
with open(os.path.join(opts.path_results,'rec_ids.pkl'),'rb') as f:
    names = pickle.load(f)

# Sort based on names to always pick same samples for medr
idxs = np.argsort(names)
names = names[idxs]

# Ranker
N = opts.medr
idxs = range(N)

glob_rank = []
glob_recall = {1:0.0,5:0.0,10:0.0}
for i in range(10):

    ids = random.sample(range(0,len(names)), N)
    im_sub = im_vecs[ids,:]
    instr_sub = instr_vecs[ids,:]
    ids_sub = names[ids]

    # if params.embedding == 'image':
    if type_embedding == 'image':
        sims = np.dot(im_sub,instr_sub.T) # for im2recipe
    else:
        sims = np.dot(instr_sub,im_sub.T) # for recipe2im

    med_rank = []
    recall = {1:0.0,5:0.0,10:0.0}

    for ii in idxs:

        name = ids_sub[ii]
        # get a column of similarities
        sim = sims[ii,:]

        # sort indices in descending order
        sorting = np.argsort(sim)[::-1].tolist()

        # find where the index of the pair sample ended up in the sorting
        pos = sorting.index(ii)

        if (pos+1) == 1:
            recall[1]+=1
        if (pos+1) <=5:
            recall[5]+=1
        if (pos+1)<=10:
            recall[10]+=1

        # store the position
        med_rank.append(pos+1)

    for i in recall.keys():
        recall[i]=recall[i]/N

    med = np.median(med_rank)
    print("median", med)

    for i in recall.keys():
        glob_recall[i]+=recall[i]
    glob_rank.append(med)

for i in glob_recall.keys():
    glob_recall[i] = glob_recall[i]/10
print("Mean median", np.average(glob_rank))
print("Recall", glob_recall)


In [ ]:
# co2 data will be big df with columns "ingredient" (idx), "co2_per_kilo"

